**IMPORTING LIBRARIES**


In [ ]:
import numpy as np
import pandas as pd
import os,re
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence  import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM,Dropout,Dense,Embedding
from tensorflow.keras.models import Sequential



**LOAD TEXT**

In [ ]:
# Define the file path where the text file is located
file_path = "/content/education.txt"  # Provide the path to your text file

# Initialize an empty list to store the content of the text file
all_text = []

# Open the file in read mode using a context manager to ensure proper handling of file resources
with open(file_path, "r") as file:
    # Read the entire content of the file and store it in the variable 'text'
    text = file.read()
    # Append the content of the file to the 'all_text' list
    all_text.append(text)

# Print the contents of the 'all_text' list, which contains the content of the file
print(all_text)

["Education is an invaluable asset that can create many opportunities for individuals in our society. It is the cornerstone of success in personal, professional, and academic lives. Education is important because it helps us to develop necessary skills and knowledge, which enables us to think critically, make informed decisions, and maximise our potential.The importance of education is undeniable, and its numerous benefits are undeniable. Education helps to provide the essential knowledge, skills, and values that are necessary for success in life. Education also helps to prepare individuals to assume positions of responsibility, as well as to think critically and develop problem-solving skills.Education creates a sense of social responsibility. It teaches people how to respect one another, as well as how to be productive members of society. Learning about history and culture can help people to better understand and appreciate the differences among different cultures, and it can also in

In [ ]:
# retrieves the first four elements of the all_text list
all_text[:4]

["Education is an invaluable asset that can create many opportunities for individuals in our society. It is the cornerstone of success in personal, professional, and academic lives. Education is important because it helps us to develop necessary skills and knowledge, which enables us to think critically, make informed decisions, and maximise our potential.The importance of education is undeniable, and its numerous benefits are undeniable. Education helps to provide the essential knowledge, skills, and values that are necessary for success in life. Education also helps to prepare individuals to assume positions of responsibility, as well as to think critically and develop problem-solving skills.Education creates a sense of social responsibility. It teaches people how to respect one another, as well as how to be productive members of society. Learning about history and culture can help people to better understand and appreciate the differences among different cultures, and it can also in

In [ ]:
# Print the length of the original 'all_text' list
print("all Text length : ", len(all_text))

# Filter out elements with the value "Unknown" from the 'all_text' list
all_text = [x for x in all_text if x != "Unknown"]

# Print the length of the 'all_text' list after filtering
print("All text Length : ", len(all_text))

all Text length :  1
All text Length :  1


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
STOPWORDS = stopwords.words("english")

In [ ]:
pip install clean

In [ ]:
def clean(text):
    # Your cleaning logic goes here
    cleaned_text = text.strip() # Stripping leading and trailing whitespaces
    return cleaned_text

# Apply the clean function to each element of all_text using list comprehension
all_text = [clean(x) for x in all_text]

**N GRAM**

In [ ]:
token= Tokenizer()

In [ ]:
# preprocessing
token.fit_on_texts(all_text)

In [ ]:
# we will use in padding  and model creation and ouptut layer ;
total_words = len(token.word_index) + 1
total_words

658

In [ ]:
line  = "stand downward slide"

In [ ]:
# converting a single line of text into a sequence of tokens
token_list = token.texts_to_sequences([line])[0]

In [ ]:
# creating n-grams
for i in range(1,len(token_list)):
    n_gram_seq = token_list[:i+1]
    print(n_gram_seq)

In [ ]:
def get_all_sequences(corpus):
    # Fit the tokenizer on the entire corpus to build the vocabulary
    token.fit_on_texts(all_text)

    # Initialize an empty list to store the generated sequences of tokens
    input_sequences = []


    for line in corpus:
        # Convert the text line into a sequence of tokens using the fitted tokenizer
        token_list = token.texts_to_sequences([line])[0]

        # Generate n-gram sequences from the token list
        for i in range(1, len(token_list)):
            # Slice the token list to create n-gram sequences of increasing lengths
            n_gram_seq = token_list[:i + 1]
            # Append each n-gram sequence to the list of input sequences
            input_sequences.append(n_gram_seq)

    return input_sequences



In [ ]:
# Call the function to generate sequences of tokens from the entire text data
all_seqs = get_all_sequences(all_text)

# Display the first 10 sequences of tokens generated from the text data
print(all_seqs[:10])

[[4, 9], [4, 9, 20], [4, 9, 20, 254], [4, 9, 20, 254, 146], [4, 9, 20, 254, 146, 17], [4, 9, 20, 254, 146, 17, 11], [4, 9, 20, 254, 146, 17, 11, 62], [4, 9, 20, 254, 146, 17, 11, 62, 95], [4, 9, 20, 254, 146, 17, 11, 62, 95, 63], [4, 9, 20, 254, 146, 17, 11, 62, 95, 63, 10]]


In [ ]:
def gen_padded_seq(corpus):
    # Find the maximum length of sequences in the corpus for padding
    maxlen = max([len(x) for x in corpus])

    # Pad the sequences to have uniform length using the maximum length found
    input_sequences = np.array(pad_sequences(corpus, maxlen=maxlen, padding='pre'))  # Prefer 'pre' with LSTM/RNNs

    # Split the sequences into predictors and labels
    predictors, label = input_sequences[:, :-1], input_sequences[:, -1]

    # Convert the labels to categorical one-hot encoding
    label = to_categorical(label, num_classes=total_words)

    # Return predictors, labels, maximum length, and the padded input sequences
    return predictors, label, maxlen, input_sequences

In [ ]:
# generate padded sequences from the all_seqs data and unpacks the returned values into four variables
predictors,label,maxlen ,input_sequences= gen_padded_seq(all_seqs)

In [ ]:
print("general text we write :",list(input_sequences[:,:-1][3]))
print("Label/Next sequences should be : ",input_sequences[:,-1][3])
print("Newly formed text : ",input_sequences[:,:-1][4])

general text we write : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

**MODEL CREATION**

In [ ]:
# Determine the length of input sequences (excluding the last token)
input_len = maxlen - 1

# Define the architecture of the neural network model
model = Sequential([
    # Add an embedding layer with input dimension equal to 'total_words', output dimension 10, and input length 'input_len'
    Embedding(total_words, 10, input_length=input_len),

    # Add an LSTM layer with 100 units
    LSTM(100),

    # Add a dropout layer to prevent overfitting, with a dropout rate of 0.2
    Dropout(0.2),

    # Add a dense layer with 'total_words' neurons and softmax activation function
    # Softmax activation is used because the output represents a probability distribution over the vocabulary
    Dense(total_words, activation='softmax')
])


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2085, 10)          6580      
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 658)               66458     
                                                                 
Total params: 117438 (458.74 KB)
Trainable params: 117438 (458.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(predictors,label,epochs=100,batch_size=32)

Epoch 1/100
66/66 [==============================] - 14s 144ms/step - loss: 6.0559 - acc: 0.0350
Epoch 2/100
66/66 [==============================] - 6s 98ms/step - loss: 5.6081 - acc: 0.0532
Epoch 3/100
66/66 [==============================] - 5s 78ms/step - loss: 5.5607 - acc: 0.0489
Epoch 4/100
66/66 [==============================] - 5s 74ms/step - loss: 5.5297 - acc: 0.0518
Epoch 5/100
66/66 [==============================] - 4s 68ms/step - loss: 5.4787 - acc: 0.0499
Epoch 6/100
66/66 [==============================] - 5s 70ms/step - loss: 5.4299 - acc: 0.0441
Epoch 7/100
66/66 [==============================] - 5s 74ms/step - loss: 5.3866 - acc: 0.0489
Epoch 8/100
66/66 [==============================] - 5s 74ms/step - loss: 5.3482 - acc: 0.0504
Epoch 9/100
66/66 [==============================] - 5s 76ms/step - loss: 5.3120 - acc: 0.0518
Epoch 10/100
66/66 [==============================] - 5s 70ms/step - loss: 5.2713 - acc: 0.0547
Epoch 11/100
66/66 [===========================

In [ ]:
def to_padded(text, input_len=input_len):
    # Convert the input text into a sequence of tokens using the tokenizer
    token_list = token.texts_to_sequences([text])[0]

    # Pad the sequence of tokens to match the specified input length
    # Use 'pre' padding to ensure consistency with model training
    padded = pad_sequences([token_list], maxlen=maxlen - 1, padding='pre')

    # Return the padded sequence
    return padded


In [ ]:
# Convert the input text "United states" into a padded sequence of tokens
text  = to_padded("United states")
text

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
## Find the index of the token with the highest probability for each sequence in the batch
def predict(padded):
    return list(np.argmax(model.predict(padded),axis=1))

In [ ]:
seq  = predict(text)
seq

1/1 [==============================] - 0s 397ms/step


[2]

In [ ]:
def reverse_to_word(seq):
    # Initialize an empty string to hold the reconstructed text
    text = ""

    # Iterate over each word-index pair in the word index dictionary of the tokenizer
    for word, index in token.word_index.items():
        # Iterate over each index in the input sequence 'seq'
        for n in seq:
            # If the index matches the current index in the loop, append the corresponding word to the 'text' string
            if n == index:
                text += " " + word
                # Break out of the inner loop once the word is found for the current index
                break

    # Return the reconstructed text
    return text


In [ ]:
# Call the function to reconstruct the sequence into text
reverse_to_word(seq)

' to'

In [ ]:
def generate_next_seq(text):
    # Convert the input text into a padded sequence of tokens
    padded = to_padded(text)

    # Predict the next token classes based on the padded sequence
    classes = predict(padded)

    # Generate the next sequence of words based on the predicted token classes
    next_sequence = reverse_to_word(classes)

    # Return the generated next sequence of words
    return next_sequence


In [ ]:
# Finally Continuous cycle for text ->  pad -> predict -> to_text ->  new text  = to_text + text
def generate_text(text,n_times):


    for i in range(n_times):
        text +=  generate_next_seq(text)

    return text


In [ ]:
print(generate_text("role",25))

1/1 [==============================] - 0s 39ms/step
role and values that is one of the challenge of the world components and innovation it is the cornerstone of personal including contributing and developing the
